In [1]:
# Import required packages
import os
import sys
#import asf_search as asf
import logging
import isce
root_logger = logging.getLogger()
root_logger.setLevel('WARNING')
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from datetime import datetime, timedelta
import time
from glob import glob
import asf_search as asf

In [2]:
# Set environment variables to call ISCE from the command line
os.environ['ISCE_HOME'] = os.path.dirname(isce.__file__)
os.environ['ISCE_ROOT'] = os.path.dirname(os.environ['ISCE_HOME'])
os.environ['ISCE_STACK'] = '/mnt/Backups/gbrench/sw/insar_tools/isce2/src/isce2/contrib/stack'
os.environ['PYTHONPATH'] = os.environ['ISCE_STACK']
os.environ['PATH'] += f":{os.environ['ISCE_STACK']}/topsStack"

In [3]:
scene_list=[
'S1A_IW_SLC__1SDV_20191004T121345_20191004T121412_029309_0354B0_F48E',
'S1A_IW_SLC__1SDV_20191016T121345_20191016T121412_029484_035ABB_373F',
'S1A_IW_SLC__1SDV_20191028T121345_20191028T121412_029659_0360C3_574E',
'S1A_IW_SLC__1SDV_20191109T121345_20191109T121412_029834_0366F3_462C',
'S1A_IW_SLC__1SDV_20191121T121344_20191121T121411_030009_036CFD_787C',
'S1A_IW_SLC__1SDV_20191203T121344_20191203T121411_030184_03730F_4C1A',
'S1A_IW_SLC__1SDV_20191215T121344_20191215T121411_030359_03791A_4B57',
'S1A_IW_SLC__1SDV_20191227T121343_20191227T121410_030534_037F23_3C47',
'S1A_IW_SLC__1SDV_20200108T121343_20200108T121410_030709_038533_C53D',
'S1A_IW_SLC__1SDV_20200120T121342_20200120T121409_030884_038B59_8828',
'S1A_IW_SLC__1SDV_20200201T121342_20200201T121409_031059_039174_1908',
'S1A_IW_SLC__1SDV_20200213T121341_20200213T121408_031234_039785_41EB',
'S1A_IW_SLC__1SDV_20200225T121341_20200225T121408_031409_039D91_8F87',
'S1A_IW_SLC__1SDV_20200308T121341_20200308T121408_031584_03A39B_6624',
'S1A_IW_SLC__1SDV_20200320T121342_20200320T121409_031759_03A9BC_08F7',
'S1A_IW_SLC__1SDV_20200401T121342_20200401T121409_031934_03AFE6_E721',
'S1A_IW_SLC__1SDV_20200413T121342_20200413T121409_032109_03B610_2EA6'
           ]

In [4]:
proc_path = '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12'
os.chdir(proc_path)

In [5]:
# # download slcs
# results = asf.granule_search(scene_list)
# results.download(path=f'{proc_path}/slc', processes=2)

In [6]:
# # download dem
# os.chdir(f'{proc_path}/dem')
# !sardem --bbox 85.9 27.1 87.6 28.6 --xrate 3 --yrate 3 --data-source COP -isce

In [7]:
# from isce.applications.gdal2isce_xml import gdal2isce_xml
# fname = './dem/earthdem_elevation.dem'
# xml_file = gdal2isce_xml(fname)

## first run to CSLCs

In [8]:
# stacksentinel to generate run files
!stackSentinel.py -s ./slc -o ./orbits -a ./aux -d ./dem/earthdem_elevation.dem -w ./work -C geometry --bbox '27.64 28.12 86.74 87.13' -W slc -C geometry --num_proc 2

Number of SAFE files found: 17
*****************************************
20191004
orbit was not found in the /mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/orbits
downloading precise or restituted orbits ...
restituted or precise orbit already exists.
*****************************************
20191016
orbit was not found in the /mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/orbits
downloading precise or restituted orbits ...
restituted or precise orbit already exists.
*****************************************
20191028
orbit was not found in the /mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/orbits
downloading precise or restituted orbits ...
restituted or precise orbit already exists.
*****************************************
20191109
orbit was not found in the /mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/orbits
downloading precise or restituted orbits ...
restituted or precise orbit already exists.
*****************************************
201

In [9]:
%%time
!sh ./work/run_files/run_01_unpack_topo_reference

Completed Parsing the Configuration file
Functions to be executed:
['Function-1', 'Function-2']
Running: Sentinel1_TOPS
['--dirname', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/slc/S1A_IW_SLC__1SDV_20191004T121345_20191004T121412_029309_0354B0_F48E.zip', '--swaths', '1 2 3', '--orbit', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/orbits/20191004/S1A_OPER_AUX_POEORB_OPOD_20210314T115943_V20191003T225942_20191005T005942.EOF', '--outdir', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference', '--auxdir', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/aux', '--bbox', '27.64 28.12 86.74 87.13', '--pol', 'vv']
Input XML files:  ['/vsizip//mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/slc/S1A_IW_SLC__1SDV_20191004T121345_20191004T121412_029309_0354B0_F48E.zip/S1A_IW_SLC__1SDV_20191004T121345_20191004T121412_029309_0354B0_F48E.SAFE/annotation/s1a-iw1-slc-vv-20191004t121347-20191004t121412-029309-0354b0-004.xml']
Input TI

In [10]:
%%time
!sh ./work/run_files/run_02_unpack_secondary_slc

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: Sentinel1_TOPS
['--dirname', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/slc/S1A_IW_SLC__1SDV_20191016T121345_20191016T121412_029484_035ABB_373F.zip', '--swaths', '1 2 3', '--orbit', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/orbits/20191016/S1A_OPER_AUX_POEORB_OPOD_20210314T154409_V20191015T225942_20191017T005942.EOF', '--outdir', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/secondarys/20191016', '--auxdir', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/aux', '--bbox', '27.64 28.12 86.74 87.13', '--pol', 'vv']
Running: Sentinel1_TOPS
['--dirname', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/slc/S1A_IW_SLC__1SDV_20191028T121345_20191028T121412_029659_0360C3_574E.zip', '--swaths', '1 2 3', '--orbit', '/mnt/Backups/gbrench/repos/fusits/nbs/i

In [11]:
%%time
!sh ./work/run_files/run_03_average_baseline

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: computeBaseline
['--reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference/', '--secondary', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/secondarys/20191016', '--baseline_file', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/baselines/20191004_20191016/20191004_20191016.txt']
Running: computeBaseline
['--reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference/', '--secondary', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/secondarys/20191028', '--baseline_file', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/baselines/20191004_20191028/20191004_20191028.txt']
minSecondary,maxSecondary 1 3
minReference,maxReference 1 3
minBurst, maxBurst:  1 3
minSecondary,maxSecondary 1 3
minReference,maxRef

In [12]:
%%time
!sh ./work/run_files/run_04_fullBurst_geo2rdr

Completed Parsing the Configuration file
Functions to be executed:
Completed Parsing the Configuration file
['Function-1']
Functions to be executed:
['Function-1']
Running: geo2rdr
Running: geo2rdr
['--secondary', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/secondarys/20191028', '--reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference', '--geom_referenceDir', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/geom_reference', '--coregSLCdir', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/coreg_secondarys/20191028', '--azimuth_misreg', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/misreg/azimuth/dates/20191028.txt', '--range_misreg', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/misreg/range/dates/20191028.txt']
['--secondary', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/secondarys/20191016', '--reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_pu

In [13]:
%%time
!sh ./work/run_files/run_05_fullBurst_resample

Completed Parsing the Configuration file
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Functions to be executed:
['Function-1']
Running: resamp_withCarrier
['--secondary', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/secondarys/20191028', '--reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference', '--coregdir', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/coreg_secondarys/20191028', '--azimuth_misreg', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/misreg/azimuth/dates/20191028.txt', '--range_misreg', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/misreg/range/dates/20191028.txt']
Running: resamp_withCarrier
['--secondary', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/secondarys/20191016', '--reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference', '--coregdir', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/a

In [14]:
%%time
!sh ./work/run_files/run_06_extract_stack_valid_region

creating  /mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/stack
checking the number of bursts in coreg_secondarys against the one in reference
all secondary images have the same number of bursts as the reference
******************
swath:  1
/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/coreg_secondarys/20191016
minSecondary,maxSecondary 1 3
minReference,maxReference 1 3
minBurst, maxBurst:  1 3
/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/coreg_secondarys/20191028
minSecondary,maxSecondary 1 3
minReference,maxReference 1 3
minBurst, maxBurst:  1 3
/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/coreg_secondarys/20191109
minSecondary,maxSecondary 1 3
minReference,maxReference 1 3
minBurst, maxBurst:  1 3
/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/coreg_secondarys/20191121
minSecondary,maxSecondary 1 3
minReference,maxReference 1 3
minBurst, maxBurst:  1 3
/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/A

In [15]:
%%time
!sh ./work/run_files/run_07_merge_reference_secondary_slc

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/stack', '--inp_reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference', '--dirname', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference', '--name_pattern', 'burst*slc', '--outfile', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/SLC/20191004/20191004.slc', '--method', 'top', '--valid_only', '--range_looks', '9', '--azimuth_looks', '3']
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 3
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 3
writing merged file to disk via gdal.Translate ...
0010..20.3040..50.6070..80.90100 
Skipping multi-looking ....
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--sta

In [16]:
%%time
!sh ./work/run_files/run_08_grid_baseline

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: baselineGrid
['--reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference/', '--secondary', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/secondarys/20191016', '--baseline_file', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/baselines/20191016/20191016']
Baseline file /mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/baselines/20191016/20191016 dims: 11L x 16P
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 
0	
Polynomial Order: 0 - by - 0 

In [17]:
# remove run files
!rm -r ./work/run_files

In [18]:
!mv ./work/coreg_secondarys ./work/coreg_secondarys_tmp
!mv ./work/merged ./work/merged_cslcs

## run to wrapped interferograms 

In [19]:
# stacksentinel to generate run files
# NOTE: do not change work directory name to anything other than "work" or things will break
!stackSentinel.py -s ./slc -o ./work/orbits -a ./aux -d ./dem/earthdem_elevation.dem -w ./work -C geometry --bbox '27.64 28.12 86.74 87.13' -W interferogram -C geometry --num_proc 1 -c 5 -z 1 -r 5  

Number of SAFE files found: 17
Number of SAFE files to be used (cover BBOX): 17
*****************************************
Number of dates : 17
List of dates : 
['20191004', '20191016', '20191028', '20191109', '20191121', '20191203', '20191215', '20191227', '20200108', '20200120', '20200201', '20200213', '20200225', '20200308', '20200320', '20200401', '20200413']
date      south      north
20191004 26.301712 28.337379
20191016 26.302294 28.337801
20191028 26.301863 28.337366
20191109 26.301846 28.337383
20191121 26.301462 28.337151
20191203 26.301331 28.337381
20191215 26.301512 28.337048
20191227 26.301704 28.337463
20200108 26.301456 28.337101
20200120 26.301537 28.337097
20200201 26.301271 28.336708
20200213 26.301449 28.336884
20200225 26.301344 28.337124
20200308 26.301584 28.337339
20200320 26.301432 28.336967
20200401 26.301035 28.336603
20200413 26.301346 28.337135
*****************************************
The overlap region among all dates (based on the preview kml files):
 Sou

In [20]:
!mv ./work/coreg_secondarys_tmp ./work/coreg_secondarys

In [21]:
%%time
!sh ./work/run_files/run_07_merge_reference_secondary_slc

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/stack', '--inp_reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference', '--dirname', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference', '--name_pattern', 'burst*slc', '--outfile', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/SLC/20191004/20191004.slc', '--method', 'top', '--valid_only', '--use_virtual_files', '--range_looks', '5', '--azimuth_looks', '1']
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 3
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 3
Skipping multi-looking ....
Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12

In [22]:
%%time
!sh ./work/run_files/run_08_generate_burst_igram

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: generateIgram
['--reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/reference', '--secondary', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/coreg_secondarys/20191016', '--interferogram', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/interferograms/20191004_20191016', '--interferogram_prefix', 'fine']
minSecondary,maxSecondary 1 3
minReference,maxReference 1 3
minBurst, maxBurst:  1 3
matching burst numbers:  1 1
multiply
read
No range offsets provided
matching burst numbers:  2 2
multiply
read
No range offsets provided
matching burst numbers:  3 3
multiply
read
No range offsets provided
Type:  <class 'isceobj.Sensor.TOPS.TOPSSwathSLCProduct.TOPSSwathSLCProduct'>
minSecondary,maxSecondary 1 3
minReference,maxReference 1 3
minBurst, maxBurst:  1 3
matching burst numbers:  1 1
multiply
read
No range offsets provided
matching burst numbers: 

In [23]:
%%time
!sh ./work/run_files/run_09_merge_burst_igram

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: mergeBursts
['--stack', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/stack', '--inp_reference', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/interferograms/20191004_20191016', '--dirname', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/interferograms/20191004_20191016', '--name_pattern', 'fine*int', '--outfile', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/interferograms/20191004_20191016/fine.int', '--method', 'top', '--aligned', '--valid_only', '--use_virtual_files', '--multilook', '--range_looks', '5', '--azimuth_looks', '1']
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 3
Updating the valid region of each burst to the common valid region of the stack
bursts:  1 3
multilooking 5 x 1 using isce2 for /mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/interfe

In [25]:
%%time
!sh ./work/run_files/run_10_filter_coherence

Completed Parsing the Configuration file
Functions to be executed:
['Function-1']
Running: FilterAndCoherence
['--input', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/interferograms/20191004_20191016/fine.int', '--filt', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/interferograms/20191004_20191016/filt_fine.int', '--coh', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/interferograms/20191004_20191016/filt_fine.cor', '--strength', '0.5', '--slc1', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/SLC/20191004/20191004.slc.full', '--slc2', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/SLC/20191016/20191016.slc.full', '--complex_coh', '/mnt/Backups/gbrench/repos/fusits/nbs/imja/agu_push/AT12/work/merged/interferograms/20191004_20191016/fine.cor', '--range_looks', '5', '--azimuth_looks', '1']
2023-11-14 20:05:29,552 - isce.tops.runFilter - INFO - Applying power-spectral filt